In [2]:
# Import Libraries
import pandas as pd
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re

In [3]:
# Load Raw Dataset
df = pd.read_csv("C:/Users/pe225/Desktop/Work Samples/Hotel Review Analysis/Hotel_Reviews.csv")
print(df.shape)
print(df.columns)
df.head(3)

(515738, 17)
Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')


,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968


In [ ]:
# # Check field name
print(df.columns.tolist())

['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date', 'Average_Score', 'Hotel_Name', 'Reviewer_Nationality', 'Negative_Review', 'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews', 'Positive_Review', 'Review_Total_Positive_Word_Counts', 'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags', 'days_since_review', 'lat', 'lng']


In [ ]:
# Remove data with missing ratings.
df = df[df["Reviewer_Score"].notnull()].copy()

In [ ]:
# # Process date formats and extract the year
df["Review_Date"] = pd.to_datetime(df["Review_Date"])
df["year"] = df["Review_Date"].dt.year

In [ ]:
# Combine positive and negative comments into one column: review_text
df["review_text"] = (
    df["Positive_Review"].replace("No Positive", "") + " " +
    df["Negative_Review"].replace("No Negative", "")
)

In [ ]:
# Extracting passenger types from tags
def get_type(tag):
    tag = str(tag).lower()
    if "family" in tag: return "Family"
    elif "couple" in tag: return "Couple"
    elif "business" in tag: return "Business"
    elif "solo" in tag: return "Solo"
    else: return "Other"

df["stay_purpose"] = df["Tags"].apply(get_type)

In [ ]:
# Added comment length field
df["review_len"] = df["review_text"].str.len()

In [ ]:
# Filter and analyze the required fields
df = df[[
    "Hotel_Name", "Reviewer_Nationality", "Reviewer_Score", "year",
    "review_text", "stay_purpose", "review_len"
]]

print("Cleaned dataset shape:", df.shape)

✅ Cleaned dataset shape: (515738, 7)


In [11]:
# Sentiment Analysis (VADER)
print("Running sentiment analysis...")

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return 0
    return analyzer.polarity_scores(text)["compound"]

df["sentiment_score"] = df["review_text"].apply(get_sentiment)

print("Sentiment scores added!")

Running sentiment analysis...
Sentiment scores added!


In [15]:
# Exploratory Checks (Quick Sanity Test)
print("\n Average sentiment and rating by traveler type:")
summary = (
    df.groupby("stay_purpose")[["sentiment_score", "Reviewer_Score"]]
      .mean()
      .round(2)
)
print(summary)


 Average sentiment and rating by traveler type:
              sentiment_score  Reviewer_Score
stay_purpose                                 
Business                 0.40            7.95
Couple                   0.52            8.51
Family                   0.48            8.34
Other                    0.52            8.51
Solo                     0.49            8.35


In [17]:
# Export Cleaned & Sentiment Data
output_path = Path(r"C:\Users\pe225\Desktop\Work Samples\Hotel Review Analysis\hotel_reviews_for_tableau.csv")

# 匯出 CSV
df.to_csv(output_path, index=False, encoding="utf-8-sig")